In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow
from pymongo import MongoClient
import pymongoarrow.monkey
import pymongoarrow.api as arrow_api
import pprint
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib


# Replace 'your_connection_string' with your MongoDB connection string
client = MongoClient('URL')
db = client.predictive_maintenance
collection = db.data_records_ts

# Show a sample document
pprint.pprint(collection.find_one())

pymongoarrow.monkey.patch_all()

# Specify the schema for the pandas DataFrame
schema = arrow_api.Schema({
    "time": pyarrow.timestamp('ms'),
    "temperature": float,
    "pressure": float,
    "vibration": float,
    "failure": int
})

# Define the aggregation pipeline to filter and sort the data based on the timestamp
pipeline = [
    {"$match": {"sensor_id": "sensor01"}},  # Filter data
    {"$sort": {"timestamp": 1}},  # Sort data by timestamp (ascending)
]

# Fetch data from MongoDB and load it into a pandas DataFrame
data = collection.aggregate_pandas_all(pipeline, schema=schema)

# Close the connection
client.close()

# Display the first 20 rows of the DataFrame
print(type(data))
print(data.head(20))


In [ ]:
# Plot
def plot_variable(variable='temperature'):
    plt.figure(figsize=(12, 6))
    
    if variable == 'temperature':
        plt.plot(data['temperature'], color='red', label='Temperature', alpha=0.7)
        plt.ylabel('Temperature')
    elif variable == 'pressure':
        plt.plot(data['pressure'], color='yellow', label='Pressure', alpha=0.7)
        plt.ylabel('Pressure')
    elif variable == 'vibration':
        plt.plot(data['vibration'], color='blue', label='Vibration', alpha=0.7)
        plt.ylabel('Vibration')
    elif variable == 'failure':
        plt.plot(data['failure'], color='green', label='failure', alpha=0.7)
        plt.ylabel('Failure')
    
    plt.xlabel('Record Index')
    plt.legend()
    plt.title(f'{variable.capitalize()} Data')
    plt.show()

# Create a toggle switch for the three variables
variable_selector = widgets.Dropdown(
    options=['temperature', 'pressure', 'vibration', 'failure'],
    value='temperature',
    description='Variable:',
)

# Display the interactive chart with a toggle switch
widgets.interact(plot_variable, variable=variable_selector)


In [ ]:
# Prepare the feature matrix (X) and the target vector (y)
X = data.drop('failure', axis=1)
X = data.drop('time', axis=1)
y = data['failure']

# Create an estimator for RFE (using Logistic Regression in this example)
estimator = LogisticRegression()

# Perform RFE to rank and select the most informative features
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X, y)

# Create a Series to map feature names to their RFE rankings
ranked_features = pd.Series(selector.ranking_, index=X.columns)

# Sort the features based on their RFE rankings
ranked_features = ranked_features.sort_values()

# Print the ranked features
print("Ranked features based on recursive feature elimination:")
print(ranked_features)

In [ ]:
# Separate features and target variable
X = data[['temperature', 'pressure', 'vibration']]
y = data['failure']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model on the testing set
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1))

# Save the model for deployment
joblib.dump(model, 'logistic_regression_model.pkl')

In [ ]:
# Input features for the model
input_data = np.array([[250, 1000, 50]])

# Predict the probability of equipment failure
loaded_model = joblib.load('logistic_regression_model.pkl')
probability = loaded_model.predict_proba(input_data)[:, 1]

print(f"Probability of equipment failure at 220 degrees: {probability[0]:.4f}")

pprint.pprint(probability)